In [1]:
import wave
import numpy as np
import os
from pathlib import Path

# Define your paths here
file1_path = "/proj/separated/full_stems/htdemucs/Unterwegs_mit_Dir-rock12_84780563693569_converted/vocals.wav"  # Change this
file2_path = "/proj/separated/full_stems/htdemucs/Unterwegs_mit_Dir-rock12_84780563693569_converted/other.wav"  # Change this
output_dir = "/proj/separated/full_stems/htdemucs/Unterwegs_mit_Dir-rock12_84780563693569_converted"  # Change this
output_filename = "test_combination.wav"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, output_filename)

print(f"File 1: {file1_path}")
print(f"File 2: {file2_path}")
print(f"Output: {output_path}")

File 1: /proj/separated/full_stems/htdemucs/Unterwegs_mit_Dir-rock12_84780563693569_converted/vocals.wav
File 2: /proj/separated/full_stems/htdemucs/Unterwegs_mit_Dir-rock12_84780563693569_converted/other.wav
Output: /proj/separated/full_stems/htdemucs/Unterwegs_mit_Dir-rock12_84780563693569_converted/test_combination.wav


In [2]:
def combine_wav_files(file1_path, file2_path, output_path):
    """
    Combines two WAV files by adding their audio samples together.
    Both files must have the same sample rate, channels, and bit depth.
    """
    with wave.open(file1_path, 'rb') as wav1, wave.open(file2_path, 'rb') as wav2:
        # Get parameters from both files
        params1 = wav1.getparams()
        params2 = wav2.getparams()
        
        # Check if parameters match
        if params1[:4] != params2[:4]:
            print("❌ WAV parameters don't match!")
            print(f"File 1: {params1}")
            print(f"File 2: {params2}")
            return False
        
        # Read audio data
        frames1 = wav1.readframes(params1.nframes)
        frames2 = wav2.readframes(params2.nframes)
        
        # Convert to numpy arrays
        dtype = np.int16 if params1.sampwidth == 2 else np.int8
        arr1 = np.frombuffer(frames1, dtype=dtype)
        arr2 = np.frombuffer(frames2, dtype=dtype)
        
        # Make sure both arrays have the same length (use shorter one)
        min_len = min(len(arr1), len(arr2))
        arr1 = arr1[:min_len]
        arr2 = arr2[:min_len]
        
        print(f"📊 Combining {min_len} samples...")
        
        # Add the audio samples together (with overflow protection)
        mixed = arr1.astype(np.int32) + arr2.astype(np.int32)
        
        # Prevent clipping by limiting to valid range
        max_val = np.iinfo(dtype).max
        min_val = np.iinfo(dtype).min
        mixed = np.clip(mixed, min_val, max_val).astype(dtype)
        
        # Write the combined audio
        with wave.open(output_path, 'wb') as out_wav:
            out_wav.setparams(params1)
            out_wav.writeframes(mixed.tobytes())
        
        print(f"✅ Combined audio saved to: {output_path}")
        return True

In [3]:
# Check if input files exist
if not os.path.exists(file1_path):
    print(f"❌ File 1 not found: {file1_path}")
elif not os.path.exists(file2_path):
    print(f"❌ File 2 not found: {file2_path}")
else:
    # Combine the files
    success = combine_wav_files(file1_path, file2_path, output_path)
    
    if success:
        print(f"🎵 Successfully combined audio files!")
        print(f"📁 Output saved to: {output_path}")
        
        # Show file size
        file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
        print(f"📊 Output file size: {file_size:.2f} MB")

📊 Combining 11201184 samples...
✅ Combined audio saved to: /proj/separated/full_stems/htdemucs/Unterwegs_mit_Dir-rock12_84780563693569_converted/test_combination.wav
🎵 Successfully combined audio files!
📁 Output saved to: /proj/separated/full_stems/htdemucs/Unterwegs_mit_Dir-rock12_84780563693569_converted/test_combination.wav
📊 Output file size: 21.36 MB
